![](attachment:./RackMultipart20220329-4-1ky0ca6_html_ab3a6516da80d7cc.jpg)

# Relatórios

Agora, chegamos no momento em que a qualidade foi aferida e as transformações necessárias para que osdados sejam úteis, já foram realizadas. Este momento conseguimos responder perguntas importantes paradirecionar esforços para melhorar a qualidade dos dados e também responder perguntas de negócio quesão essenciais para tomada de decisão.

In [1]:
!pip install pyspark
!pip install findspark
!pip install great_expectations

You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 4.9 MB 34.6 MB/s            
     |████████████████████████████████| 67 kB 8.0 MB/s             
     |████████████████████████████████| 3.6 MB 55.0 MB/s            
     |████████████████████████████████| 109 kB 59.8 MB/s            
     |████████████████████████████████| 812 kB 36.8 MB/s            
     |████████████████████████████████| 55 kB 7.3 MB/s             
     |████████████████████████████████| 570 kB 38.6 MB/s            
     |████████████████████████████████| 74 kB 4.0 MB/s             
     |████████████████████████████████| 339 kB 45.7 MB/s            
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.7
    Uninstalling pyparsing-3.0.7:
      Successfully uninstalled pypars

In [2]:
!printf 'yes' | great_expectations init

Using v3 (Batch Request) API

  ___              _     ___                  _        _   _
 / __|_ _ ___ __ _| |_  | __|_ ___ __  ___ __| |_ __ _| |_(_)___ _ _  ___
| (_ | '_/ -_) _` |  _| | _|\ \ / '_ \/ -_) _|  _/ _` |  _| / _ \ ' \(_-<
 \___|_| \___\__,_|\__| |___/_\_\ .__/\___\__|\__\__,_|\__|_\___/_||_/__/
                                |_|
             ~ Always know what to expect from your data ~

This looks like an existing project that appears complete! You are ready to roll.



In [2]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, StructField, StructType, IntegerType, FloatType

from datetime import datetime as dt

In [3]:
sc = SparkContext()

spark = (SparkSession.builder.master('local[2]').appName("Relatórios - wesley"))

In [5]:
from ruamel import yaml

import great_expectations as ge
from great_expectations.core.batch import BatchRequest, RuntimeBatchRequest

In [6]:
context = ge.get_context()

/opt/python/envs/default/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# Expressoes regulares comuns
REGEX_ALPHA    = r'[a-zA-Z]+'
REGEX_INTEGER  = r'[0-9]+'
REGEX_FLOAT    = r'[0-9]+\.[0-9]+'
REGEX_ALPHANUM = r'[0-9a-zA-Z]+'
REGEX_EMPTY_STR= r'[\t ]+$'
REGEX_SPECIAL  = r'[!@#$%&*\(\)_]+'
REGEX_NNUMBER  = r'^N[1-9][0-9]{2,3}([ABCDEFGHJKLMNPRSTUVXWYZ]{1,2})'
REGEX_NNUMBER_INVALID = r'(N0.*$)|(.*[IO].*)'
REGEX_TIME_FMT = r'^(([0-1]?[0-9])|(2[0-3]))([0-5][0-9])$'

In [5]:
import re

def check_empty_column(col):
    return (F.col(col).isNull() | (F.col(col) == '') | (F.col(col).rlike(REGEX_EMPTY_STR) ))

In [6]:
def add_test_rows_for_airports(data, _format):
    """ Adiciona linhas para testar regras de negócio"""
    
    values = [
        #faa    name  lat       lon               alt       tz    dst
        (None , None, None    , None            , None    , None, None),
        (''   , ''  , ''      , ''              , ''      , ''  , ''  ),
        ('   ', None, '12O.12', '-80.Aa6195833' , '1044Aa', '-2x', '34'),
        ('AAA', None, '12O.12', '-80.Aa6195833' , '1044Aa', '-2x', '34'),
        ('222', None, None    , '-80.Aa6195833' , '-100'  , '-14', 'K'),
        ('__!', None, None    , '-80.Aa6195833' , '-100'  , '-14', 'K')
    ]
 
    if _format == "df":
        return spark.getOrCreate().createDataFrame(values, data.columns).union(data)
    elif _format == "rdd":
        return sc.parallelize(tuple(values)).union(data)

def add_test_rows_for_flights(data, _format):
    """ Adiciona linhas para testar regras de negócio"""
    
    values = [
        #|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
        (None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None),
        (12,None,None,None,12,None,None,None,12,None,None,None,12,None,None,12,12),
        (13,12,1,123,None,12,None,12,None,12,None,12,None,12,None,12,22),
        (1950,13,29,2244,None,None,12,None,None,None,12,None,None,None,12,None,None),
        (1950,2,29,234,None,None,12,None,None,None,12,None,None,None,12,None,None),
        (2022,8,27,1345,None,None,12,None,None,None,12,None,None,None,12,None,None),
    ]
 
    if _format == "df":
        return spark.getOrCreate().createDataFrame(values, data.columns).union(data)
    elif _format == "rdd":
        return sc.parallelize(tuple(values)).union(data)

In [7]:
# Lendo arquivo parquet

flights_qa = (spark.getOrCreate().read
                   .format("parquet")
                   .option("header","True")
                   .load("./parquet_files/qa_outputs/qa_flights.parquet"))


# Lendo arquivo parquet
planes_qa = (spark.getOrCreate().read
                   .format("parquet")
                   .option("header","True")
                   .load("./parquet_files/qa_outputs/planes_qa.parquet"))

# Lendo arquivo parquet
airports_qa = (spark.getOrCreate().read
                   .format("parquet")
                   .option("header","True")
                   .load("./parquet_files/qa_outputs/airport_qa.parquet"))

In [11]:
# planes_qa = spark.getOrCreate().read.parquet("parquet_files/qa_outputs/qa_planes.parquet")
# flights_qa = spark.getOrCreate().read.parquet("parquet_files/qa_outputs/qa_flights.parquet")
# airports_qa = spark.getOrCreate().read.parquet("parquet_files/qa_outputs/qa_airport.parquet")
# Criacao das visões temporarias
flights_qa.createOrReplaceTempView('flights')
planes_qa.createOrReplaceTempView('planes')
airports_qa.createOrReplaceTempView('airports')

In [19]:
planes_qa.show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+-------+-------+----------------+--------+----------+--------+--------+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_tailnum|qa_year|qa_type| qa_manufacturer|qa_model|qa_engines|qa_seats|qa_speed|qa_engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+-------+-------+----------------+--------+----------+--------+--------+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         S|   1998|      M|AIRBUS INDUSTRIE|A320-214|         2|       F|       M|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         S|   1999|      M|AIRBUS INDUSTRIE|A320-214|         2|       F|       M|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null

In [19]:
flights_qa.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+-----------------+--------------+---------------+----------------+----------+---------+--------------+-----------+-----------+----------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|qa_year_month_day|qa_hour_minute|qa_dep_arr_time|qa_dep_arr_delay|qa_carrier|qa_flight|qa_origin_dest|qa_air_time|qa_distance|qa_airtime|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+-----------------+--------------+---------------+----------------+----------+---------+--------------+-----------+-----------+----------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|             null|          null|           null|            null|        VX|     1780|          n

In [23]:
airports_qa.show()

+-------+--------------------+---------+-----------+-------+------+-------+----------+--------------------+----------+-----------+----------+---------+----------+
|air_faa|            air_name|  air_lat|    air_lon|air_alt|air_tz|air_dst|air_qa_faa|         air_qa_name|air_qa_lat| air_qa_lon|air_qa_alt|air_qa_tz|air_qa_dst|
+-------+--------------------+---------+-----------+-------+------+-------+----------+--------------------+----------+-----------+----------+---------+----------+
|    04G|   Lansdowne Airport|41.130474|  -80.61958|   1044|  -5.0|      A|       04G|   Lansdowne Airport| 41.130474|  -80.61958|      1044|     -5.0|         A|
|    06A|Moton Field Munic...| 32.46057|  -85.68003|    264|  -5.0|      A|       06A|Moton Field Munic...|  32.46057|  -85.68003|       264|     -5.0|         A|
|    06C| Schaumburg Regional| 41.98934|  -88.10124|    801|  -6.0|      A|       06C| Schaumburg Regional|  41.98934|  -88.10124|       801|     -6.0|         A|
|    06N|     Randall 

In [8]:
# Renomeando colunas
flights_qa = flights_qa.select(*(F.col(x).alias('fl_' + x ) for x in flights_qa.columns))
planes_qa = planes_qa.select(*(F.col(x).alias('pl_' + x ) for x in planes_qa.columns))
airports_qa2 = airports_qa.select(*(F.col(x).alias('air2_' + x ) for x in airports_qa.columns))
airports_qa = airports_qa.select(*(F.col(x).alias('air_' + x ) for x in airports_qa.columns))

In [14]:
flights_qa.show()

+-------+--------+------+-----------+------------+-----------+------------+----------+----------+---------+---------+----------+-----------+-----------+-------+---------+--------------------+-----------------+------------------+-------------------+-------------+------------+-----------------+--------------+--------------+-------------+
|fl_year|fl_month|fl_day|fl_dep_time|fl_dep_delay|fl_arr_time|fl_arr_delay|fl_carrier|fl_tailnum|fl_flight|fl_origin|fl_destiny|fl_air_time|fl_distance|fl_hour|fl_minute|fl_qa_year_month_day|fl_qa_hour_minute|fl_qa_dep_arr_time|fl_qa_dep_arr_delay|fl_qa_carrier|fl_qa_flight|fl_qa_origin_dest|fl_qa_air_time|fl_qa_distance|fl_qa_airtime|
+-------+--------+------+-----------+------------+-----------+------------+----------+----------+---------+---------+----------+-----------+-----------+-------+---------+--------------------+-----------------+------------------+-------------------+-------------+------------+-----------------+--------------+--------------+-

/opt/python/envs/default/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
# airport_df.join(flights_qa).where(
#                     (airport_df['faa'] == flights_qa['origin'])
#             )\
#            .join(planes_qa).where(planes_qa['tailnum'] == flights_qa['tailnum'])\
#            .show()

"""
airports_qa.faa-flights_qa.origin,
airports_qa.faa-flights_qa.dest,
planes_qa.tailnum-flights_qa.tailnum2. 
"""

joined_dfs   = spark.getOrCreate().read.parquet("parquet_files/newjoin.parquet")
myjoined_dfs = spark.getOrCreate().read.parquet("parquet_files/joined_data_new.parquet")
transformation_c_dfs = spark.getOrCreate().read.parquet("parquet_files/transformed_join.parquet")

In [10]:
print(joined_dfs.count())
print(myjoined_dfs.count())

14025
11395




# Perguntas para qualidade

## 1. Crie uma base de dados unificando os 3 datasets de qualidade(airports, flights e planes)

considere as seguintes chaves em comum:

- airports\_qa.faa

- airports\_qa.faa-flights\_qa.dest,

- planes\_qa.tailnum-flights\_qa.tailnum

In [11]:
qa_challenge_df = flights_qa.join(airports_qa, (flights_qa.fl_origin == airports_qa.air_faa) ,'left')\
                            .join(airports_qa2, (flights_qa.fl_destiny  == airports_qa2.air2_faa),'left')\
                            .join(planes_qa, (flights_qa.fl_tailnum == planes_qa.pl_tailnum),'left')

In [12]:
qa_challenge_df.show()

+-------+--------+------+-----------+------------+-----------+------------+----------+----------+---------+---------+----------+-----------+-----------+-------+---------+--------------------+-----------------+------------------+-------------------+-------------+------------+-----------------+--------------+--------------+-------------+-------+-------------------+---------+---------+-------+------+-------+----------+-------------------+----------+----------+----------+---------+----------+--------+--------------------+---------+-----------+--------+-------+--------+-----------+--------------------+-----------+-----------+-----------+----------+-----------+----------+-------+--------------------+---------------+-----------+----------+--------+--------+---------+-------------+----------+----------+------------------+-----------+-------------+-----------+-----------+------------+
|fl_year|fl_month|fl_day|fl_dep_time|fl_dep_delay|fl_arr_time|fl_arr_delay|fl_carrier|fl_tailnum|fl_flight|fl_o

In [13]:
all_cols_join = spark.getOrCreate().read.parquet("parquet_files/all_join.parquet")

In [14]:
joined_dfs.show()
joined_dfs.createOrReplaceTempView('reports_view')

+----+----+----+------+---+----+---+------+-------+------+------+------+-----+------+
| faa|name| lat|   lon|alt|  tz|dst|qa_faa|qa_name|qa_lat|qa_lon|qa_alt|qa_tz|qa_dst|
+----+----+----+------+---+----+---+------+-------+------+------+------+-----+------+
|2014|  12| 8.0| 658.0| -7| 935| -5|     F|   null|     I|  null|     I|    A|     C|
|2014|   1|22.0|1040.0|  5|1505|  5|     F|   null|     I|  null|     A|    A|     C|
|2014|   3| 9.0|1443.0| -2|1652|  2|     F|   null|     I|  null|     I|    A|     C|
|2014|   4| 9.0|1705.0| 45|1839| 34|     F|   null|     I|  null|     A|    A|     C|
|2014|   3| 9.0| 754.0| -1|1015|  1|     F|   null|     I|  null|     I|    A|     C|
|2014|   1|15.0|1037.0|  7|1352|  2|     F|   null|     I|  null|     A|    A|     C|
|2014|   7| 2.0| 847.0| 42|1041| 51|     F|   null|     I|  null|     A|    A|     C|
|2014|   5|12.0|1655.0| -5|1842|-18|     F|   null|     I|  null|     I|    A|     C|
|2014|   4|19.0|1236.0| -4|1508| -7|     F|   null|   

## 2. Paracada coluna, calcule as ocorrencias de cada indicador dequalidade: *M*,*F*,*I*,*S*,*T*?

In [15]:
from collections import Counter
import json

@F.udf(StringType())
def count_occurances(row):
    return json.dumps(dict(Counter(row.split(',') ) ) )

joined_dfs.withColumn('concat', F.concat_ws(',', all_cols_join.dst, all_cols_join.qa_dst))\
  .withColumn('counts', count_occurances('concat') ).show(2, False)

joined_dfs.printSchema( )

AnalysisException: AnalysisException: Resolved attribute(s) dst#1183,qa_dst#1190 missing from faa#260,name#261,lat#262,lon#263,alt#264,tz#265,dst#266,qa_faa#267,qa_name#268,qa_lat#269,qa_lon#270,qa_alt#271,qa_tz#272,qa_dst#273 in operator !Project [faa#260, name#261, lat#262, lon#263, alt#264, tz#265, dst#266, qa_faa#267, qa_name#268, qa_lat#269, qa_lon#270, qa_alt#271, qa_tz#272, qa_dst#273, concat_ws(,, dst#1183, qa_dst#1190) AS concat#1342]. Attribute(s) with the same name appear in the operation: dst,qa_dst. Please check if the right attribute(s) are used.;
!Project [faa#260, name#261, lat#262, lon#263, alt#264, tz#265, dst#266, qa_faa#267, qa_name#268, qa_lat#269, qa_lon#270, qa_alt#271, qa_tz#272, qa_dst#273, concat_ws(,, dst#1183, qa_dst#1190) AS concat#1342]
+- Relation [faa#260,name#261,lat#262,lon#263,alt#264,tz#265,dst#266,qa_faa#267,qa_name#268,qa_lat#269,qa_lon#270,qa_alt#271,qa_tz#272,qa_dst#273] parquet


In [16]:
joined_dfs.select([joined_dfs.groupBy(F.col(c).startswith(c).alias(c)).count().show() for c in joined_dfs.columns if 'qa' in c]).show()

+------+-----+
|qa_faa|count|
+------+-----+
| false|10001|
|  null| 4024|
+------+-----+

+-------+-----+
|qa_name|count|
+-------+-----+
|   null|14025|
+-------+-----+

+------+-----+
|qa_lat|count|
+------+-----+
| false|14025|
+------+-----+

+------+-----+
|qa_lon|count|
+------+-----+
| false| 4179|
|  null| 9846|
+------+-----+

+------+-----+
|qa_alt|count|
+------+-----+
| false|14025|
+------+-----+

+-----+-----+
|qa_tz|count|
+-----+-----+
|false|14025|
+-----+-----+

+------+-----+
|qa_dst|count|
+------+-----+
| false|12553|
|  null| 1472|
+------+-----+



TypeError: TypeError: Invalid argument, not a string or column: None of type <class 'NoneType'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [17]:
# print(all_cols_join.columns)
# all_cols_join.select('*').show()
# for c in all_cols_join.columns:
# all_cols_join.select(F.count(F.when(F.col(c) == 'M', 1)) ).show()

for i in ['M', 'F', 'I', 'S', 'T']:
    print(i)
    joined_dfs.select([F.count(F.when(F.col(c) == i, 1)).alias(c) for c in joined_dfs.columns]).show(vertical=True)
print('-'*200)
for i in ['M', 'F', 'I', 'S', 'T']:
    print(i)
    transformation_c_dfs.select([F.count(F.when(F.col(c) == i, 1)).alias(c) for c in transformation_c_dfs.columns]).show(vertical=True)

M
-RECORD 0-------
 faa     | 0    
 name    | 0    
 lat     | 0    
 lon     | 0    
 alt     | 0    
 tz      | 0    
 dst     | 0    
 qa_faa  | 0    
 qa_name | 0    
 qa_lat  | 2628 
 qa_lon  | 2676 
 qa_alt  | 2675 
 qa_tz   | 55   
 qa_dst  | 0    

F
-RECORD 0--------
 faa     | 0     
 name    | 0     
 lat     | 0     
 lon     | 0     
 alt     | 0     
 tz      | 0     
 dst     | 0     
 qa_faa  | 10001 
 qa_name | 0     
 qa_lat  | 0     
 qa_lon  | 0     
 qa_alt  | 0     
 qa_tz   | 0     
 qa_dst  | 0     

I
-RECORD 0--------
 faa     | 0     
 name    | 0     
 lat     | 0     
 lon     | 0     
 alt     | 0     
 tz      | 0     
 dst     | 0     
 qa_faa  | 0     
 qa_name | 0     
 qa_lat  | 11397 
 qa_lon  | 1503  
 qa_alt  | 5896  
 qa_tz   | 4083  
 qa_dst  | 0     

S
-RECORD 0------
 faa     | 0   
 name    | 0   
 lat     | 0   
 lon     | 0   
 alt     | 0   
 tz      | 0   
 dst     | 0   
 qa_faa  | 0   
 qa_name | 0   
 qa_lat  | 0   
 qa_lon  | 0   
 q

In [18]:
# Com List Comprehension
qa_challenge_df.select([qa_challenge_df.groupBy(F.col(c).startswith(c).alias(c)).count().show() for c in qa_challenge_df.columns if 'qa' in c])

+--------------------+-----+
|fl_qa_year_month_day|count|
+--------------------+-----+
|                null|10000|
+--------------------+-----+

+-----------------+-----+
|fl_qa_hour_minute|count|
+-----------------+-----+
|            false|   48|
|             null| 9952|
+-----------------+-----+

+------------------+-----+
|fl_qa_dep_arr_time|count|
+------------------+-----+
|             false|   55|
|              null| 9945|
+------------------+-----+

+-------------------+-----+
|fl_qa_dep_arr_delay|count|
+-------------------+-----+
|              false|   75|
|               null| 9925|
+-------------------+-----+

+-------------+-----+
|fl_qa_carrier|count|
+-------------+-----+
|        false|10000|
+-------------+-----+

+------------+-----+
|fl_qa_flight|count|
+------------+-----+
|       false|10000|
+------------+-----+

+-----------------+-----+
|fl_qa_origin_dest|count|
+-----------------+-----+
|             null|10000|
+-----------------+-----+

+--------------+-

TypeError: TypeError: Invalid argument, not a string or column: None of type <class 'NoneType'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

## 3. Qual variável apresenta maior número de dados faltantes?

In [60]:
i, most_F = float('-inf'), ''
results = []

for c in qa_challenge_df.columns:
    if 'qa' in c:
        results.append(qa_challenge_df.filter(~check_empty_column(c) & (F.col(c) =='M')).groupBy(F.substring(c, 1, 1).alias(c)).count().toPandas())

for _row in results:
    c = _row.columns[0]
    if _row.values.size:
        v = _row.values[0]
        if v[1] > i:
            i = v[1]
            most_F = c

print(most_F, ' -> ' ,i)

pl_qa_type  ->  9444



## 4. Qual variável apresenta maior número de formatos errados?

In [99]:
i, most_F = float('-inf'), ''
results = []
for c in qa_challenge_df.columns:
    if 'qa' in c:
        results.append(qa_challenge_df.filter((F.col(c)!='') & (F.col(c) =='F')).groupBy(F.substring(c, 1, 1).alias(c)).count().toPandas())

for _row in results:
    c = _row.columns[0]
    if _row.values.size:
        v = _row.values[0]
        if v[1] > i:
            i = v[1]
            most_F = c

print(most_F, ' -> ' ,i)

air_qa_faa  ->  10000



## 5. Qual variável apresenta maior número de fora do intervalo?

In [100]:
results = []
## 5. Qual variável apresenta maior número de formatos errados?
for c in qa_challenge_df.columns:
    if 'qa' in c:
        results.append(qa_challenge_df.filter((F.col(c)!='') & (F.col(c) =='I')).groupBy(F.substring(c, 1, 1).alias(c)).count().toPandas())

In [101]:
i, most_F = float('-inf'), '' 
for _row in results:
    c = _row.columns[0]
    if _row.values.size:
        v = _row.values[0]
        if v[1] > i:
            i = v[1]
            most_F = c

print(most_F, ' -> ' ,i)

pl_qa_year  ->  8



# Perguntasparanegócio

## 1. Crie uma base de dados unificando os 3 datasets transformados

(airports\_proc.parquet, planes\_proc.parquet,flights\_proc.parquet) considerando as seguintes chaves em comum:

In [21]:
# Lendo arquivo parquet

flights_transform = (spark.getOrCreate().read
                          .format("parquet")
                          .option("header","True")
                          .load("./parquet_files/transformation_proc/flights_proc.parquet"))


# Lendo arquivo parquet
planes_transform = (spark.getOrCreate().read
                         .format("parquet")
                         .option("header","True")
                         .load("./parquet_files/transformation_proc/planes_proc.parquet"))

# Lendo arquivo parquet
airports_transform = (spark.getOrCreate().read
                           .format("parquet")
                           .option("header","True")
                           .load("./parquet_files/transformation_proc/airports_proc.parquet"))

In [22]:
# Renomeando colunas
flights_transform = flights_transform.select(*(F.col(x).alias('fl_' + x ) for x in flights_transform.columns))
planes_transform = planes_transform.select(*(F.col(x).alias('pl_' + x ) for x in planes_transform.columns))
airports_transform2 = airports_transform.select(*(F.col(x).alias('air2_' + x ) for x in airports_transform.columns))
airports_transform = airports_transform.select(*(F.col(x).alias('air_' + x ) for x in airports_transform.columns))

In [23]:
flights_transform.show()

+-----------+-----------+------------+------------+----------+----------+---------+---------+-------+-----------+-----------+-------------------+---------------------+--------------------+----------------+-------------+---------------------+
|fl_dep_time|fl_arr_time|fl_dep_delay|fl_arr_delay|fl_carrier|fl_tailnum|fl_flight|fl_origin|fl_dest|fl_air_time|fl_distance|    fl_dep_datetime|fl_air_time_projected|fl_air_time_expected|fl_haul_duration|fl_dep_season|fl_dep_delay_category|
+-----------+-----------+------------+------------+----------+----------+---------+---------+-------+-----------+-----------+-------------------+---------------------+--------------------+----------------+-------------+---------------------+
|        658|        935|          -7|          -5|        VX|    N846VA|     1780|      SEA|    LAX|        132|        954|2014-12-08 08:58:00|                  115|                 126|      SHORT-HAUL|         FALL|          ANTECIPATED|
|       1040|       1505|       

In [24]:
transform_proc_df = flights_transform.join(airports_transform,  (flights_transform.fl_origin == airports_transform.air_faa) ,'left')\
                                     .join(airports_transform2, (flights_transform.fl_dest  == airports_transform2.air2_faa),'left')\
                                     .join(planes_transform,    (flights_transform.fl_tailnum == planes_transform.pl_tailnum),'left')

In [25]:
transform_proc_df.show()

+-----------+-----------+------------+------------+----------+----------+---------+---------+-------+-----------+-----------+-------------------+---------------------+--------------------+----------------+-------------+---------------------+-------+-------------------+---------+---------+-------+------+-------+-------------+--------+------------+------------------+--------+--------------------+---------+-----------+--------+-------+--------+-------------+---------+-------------+-------------------+----------+-------+---------+---------------+-----------+----------+--------+--------+---------+-----------+------+--------------+
|fl_dep_time|fl_arr_time|fl_dep_delay|fl_arr_delay|fl_carrier|fl_tailnum|fl_flight|fl_origin|fl_dest|fl_air_time|fl_distance|    fl_dep_datetime|fl_air_time_projected|fl_air_time_expected|fl_haul_duration|fl_dep_season|fl_dep_delay_category|air_faa|           air_name|  air_lat|  air_lon|air_alt|air_tz|air_dst|   air_region|air_type|air_military|air_administration



## 2. Quantos aeroportos existem em cadaregion?

In [34]:
# transform_proc_df.select(F.countDistinct('air_name')).groupBy('air_re   gion').orderBy('region').show() | #.agg(F.expr('count(distinct air_name)').alias("Aeroportos por regiao")).show()

transform_proc_df.groupBy(F.col('air2_region').alias('regiões')).agg(F.expr('count(distinct air2_name)').alias('aeroportos por região')).show()

+-------------+---------------------+
|      regiões|aeroportos por região|
+-------------+---------------------+
|       ALASKA|                    9|
|MAINLAND-EAST|                   24|
|MAINLAND-WEST|                   36|
+-------------+---------------------+




## 3. Qual é a maior diferença de altitude dos voos? Considere:max(alt\_origin - alt\_dest)

In [82]:
transform_proc_df.agg(F.max(F.col('air_alt')-F.col('air2_alt')).alias("Maior diferença de altiude (em pés)")).show()  

+-----------------------------------+
|Maior diferença de altiude (em pés)|
+-----------------------------------+
|                                429|
+-----------------------------------+



## 4. Qual é o atraso médio?

In [53]:
transform_proc_df.groupBy('fl_carrier').agg(F.avg('fl_arr_delay').alias('Atraso médio das chegadas (em minutos)')).show()
transform_proc_df.groupBy('fl_carrier').agg(F.avg('fl_dep_delay').alias('Atraso médio de partida (em minutos)')).show()

+----------+-------------------------+
|fl_carrier|Atraso médio nas chegadas|
+----------+-------------------------+
|        UA|        3.180780209324453|
|        AA|        6.437759336099585|
|        B6|        2.616822429906542|
|        DL|         1.38909426987061|
|        OO|       1.8887015177065767|
|        F9|        5.392265193370166|
|        US|      -1.4713896457765667|
|        HA|        2.219178082191781|
|        AS|       0.6213002114164905|
|        VX|        1.564516129032258|
|        WN|        6.006456241032999|
+----------+-------------------------+

+----------+-----------------------+
|fl_carrier|Atraso médio nas saídas|
+----------+-----------------------+
|        UA|       10.0313986679353|
|        AA|     11.402489626556017|
|        B6|      6.827102803738318|
|        DL|      6.435304990757856|
|        OO|     3.4451939291736933|
|        F9|      6.480662983425415|
|        US|      2.980926430517711|
|        HA|      2.958904109589041|
|      



## 5. Qual é o atraso médio para cada região?

In [61]:
transform_proc_df.groupBy(F.col('air2_region').alias('regiões')).agg(F.ceil(F.avg('fl_arr_delay')).alias('Atraso médio de chegada (em minutos)')).show()
transform_proc_df.groupBy(F.col('air2_region').alias('regiões')).agg(F.ceil(F.avg('fl_dep_delay')).alias('Atraso médio de partida (em minutos)')).show()

+-------------+-----------------------+
|      regiões|Atraso médio de chegada|
+-------------+-----------------------+
|       ALASKA|                      0|
|MAINLAND-EAST|                      3|
|MAINLAND-WEST|                      3|
+-------------+-----------------------+

+-------------+-----------------------+
|      regiões|Atraso médio de partida|
+-------------+-----------------------+
|       ALASKA|                      5|
|MAINLAND-EAST|                      8|
|MAINLAND-WEST|                      6|
+-------------+-----------------------+



In [83]:
transform_proc_df.groupBy(F.col('air_region').alias('regiões')).agg(F.ceil(F.avg('fl_arr_delay')).alias('Atraso médio de chegada (em minutos)')).show()
transform_proc_df.groupBy(F.col('air_region').alias('regiões')).agg(F.ceil(F.avg('fl_dep_delay')).alias('Atraso médio de partida (em minutos)')).show()

+-------------+------------------------------------+
|      regiões|Atraso médio de chegada (em minutos)|
+-------------+------------------------------------+
|MAINLAND-WEST|                                   3|
+-------------+------------------------------------+

+-------------+------------------------------------+
|      regiões|Atraso médio de partida (em minutos)|
+-------------+------------------------------------+
|MAINLAND-WEST|                                   7|
+-------------+------------------------------------+



In [85]:
transform_proc_df.groupBy(F.col('air2_region').alias('regiões')).agg(F.ceil(F.avg('fl_arr_delay')).alias('Atraso médio de chegada (em minutos)')).show()
transform_proc_df.groupBy(F.col('air2_region').alias('regiões')).agg(F.ceil(F.avg('fl_dep_delay')).alias('Atraso médio de partida (em minutos)')).show()

+-------------+------------------------------------+
|      regiões|Atraso médio de chegada (em minutos)|
+-------------+------------------------------------+
|       ALASKA|                                   0|
|MAINLAND-EAST|                                   3|
|MAINLAND-WEST|                                   3|
+-------------+------------------------------------+

+-------------+------------------------------------+
|      regiões|Atraso médio de partida (em minutos)|
+-------------+------------------------------------+
|       ALASKA|                                   5|
|MAINLAND-EAST|                                   8|
|MAINLAND-WEST|                                   6|
+-------------+------------------------------------+




## 6. Qual é o atraso acumulado por ano?

In [81]:
transform_proc_df.groupBy(F.year(F.col('fl_dep_datetime')).alias('Ano')).agg(F.sum('fl_dep_delay').alias('Atraso de partida acumulado (em minutos)')).show()
transform_proc_df.groupBy(F.year(F.col('fl_dep_datetime')).alias('Ano')).agg(F.sum('fl_arr_delay').alias('Atraso de chegada acumulado (em minutos)'))  .show()

+----+----------------------------------------+
| Ano|Atraso de partida acumulado (em minutos)|
+----+----------------------------------------+
|2015|                                      -3|
|2014|                                   60398|
+----+----------------------------------------+

+----+----------------------------------------+
| Ano|Atraso de chegada acumulado (em minutos)|
+----+----------------------------------------+
|2015|                                       9|
|2014|                                   22353|
+----+----------------------------------------+




## 7. Qual é o atraso acumulado por ano para cada região?

In [94]:
transform_proc_df.groupBy(F.year(F.col('fl_dep_datetime')).alias('Ano'), F.col('air_region').alias('Região')).agg((F.sum('fl_dep_delay') + F.sum('fl_arr_delay')).alias('Atraso origem acumulado (em minutos)')).show()
transform_proc_df.groupBy(F.year(F.col('fl_dep_datetime')).alias('Ano'), F.col('air2_region').alias('Região')).agg((F.sum('fl_dep_delay') + F.sum('fl_arr_delay')).alias('Atraso destino acumulado (em minutos)')).show()

+----+-------------+------------------------------------+
| Ano|       Região|Atraso origem acumulado (em minutos)|
+----+-------------+------------------------------------+
|2015|MAINLAND-WEST|                                   6|
|2014|MAINLAND-WEST|                               82751|
+----+-------------+------------------------------------+

+----+-------------+-------------------------------------+
| Ano|       Região|Atraso destino acumulado (em minutos)|
+----+-------------+-------------------------------------+
|2014|       ALASKA|                                 4435|
|2014|MAINLAND-EAST|                                21618|
|2015|MAINLAND-WEST|                                    6|
|2014|MAINLAND-WEST|                                56698|
+----+-------------+-------------------------------------+




## 8. Qual é o tempo de voo médio?

In [86]:
transform_proc_df.agg(F.ceil(F.avg('fl_air_time')).alias('Tempo médio de vôo (em minutos)')).show()

+-------------------------------+
|Tempo médio de vôo (em minutos)|
+-------------------------------+
|                            153|
+-------------------------------+




## 9. Qual é o tempo de voo médio para cada região?

In [87]:
transform_proc_df.groupBy(F.col('air2_region').alias('regiões')).agg(F.ceil(F.avg('fl_air_time')).alias('Tempo médio de vôo (em minutos)')).show()

+-------------+-------------------------------+
|      regiões|Tempo médio de vôo (em minutos)|
+-------------+-------------------------------+
|       ALASKA|                            228|
|MAINLAND-EAST|                            238|
|MAINLAND-WEST|                            116|
+-------------+-------------------------------+




## 10. Qual é o tempo de voo médio para cada rota (origem e destino)?

In [91]:
transform_proc_df.groupBy(F.col('fl_origin').alias('Origem'), F.col('fl_dest').alias('Destino')).agg(F.ceil(F.avg('fl_air_time')).alias('Tempo médio de vôo (em minutos)')).show(200)

+------+-------+-------------------------------+
|Origem|Destino|Tempo médio de vôo (em minutos)|
+------+-------+-------------------------------+
|   SEA|    RNO|                             75|
|   SEA|    DTW|                            220|
|   SEA|    CLE|                            234|
|   SEA|    LAX|                            127|
|   PDX|    SEA|                             35|
|   SEA|    BLI|                             23|
|   PDX|    IAH|                            214|
|   PDX|    PHX|                            131|
|   SEA|    SLC|                             89|
|   SEA|    SBA|                            119|
|   SEA|    BWI|                            270|
|   PDX|    IAD|                            268|
|   PDX|    SFO|                             86|
|   SEA|    KOA|                            348|
|   PDX|    MCI|                            175|
|   SEA|    SJC|                            104|
|   SEA|    ABQ|                            143|
|   SEA|    SAT|    


## 11. Qual é o tempo de voo acumulado por ano?

In [95]:
transform_proc_df.groupBy(F.year(F.col('fl_dep_datetime')).alias('Ano')).agg(F.ceil(F.avg('fl_air_time')).alias('Tempo médio de vôo (em minutos)')).show()

+----+-------------------------------+
| Ano|Tempo médio de vôo (em minutos)|
+----+-------------------------------+
|2015|                             43|
|2014|                            153|
+----+-------------------------------+




## 12. Qual é o tempo de voo acumulado para cada região?

In [98]:
transform_proc_df.groupBy(F.col('air2_region').alias('regiões')).agg(F.sum('fl_air_time').alias('Tempo total de vôo (em minutos)')).show()

+-------------+-------------------------------+
|      regiões|Tempo total de vôo (em minutos)|
+-------------+-------------------------------+
|       ALASKA|                         230602|
|MAINLAND-EAST|                         508344|
|MAINLAND-WEST|                         789679|
+-------------+-------------------------------+




## 13. Qual é a distância de voo médio?


## 14. Qual é a distância de voo médio para cada região?


## 15. Qual é a distância de voo médio para cada rota (origem e destino)?


## 16. Qual é a distância de voo acumulada por ano?


## 17. Qual é a distância de voo acumulada para cada região?


## 18. Qual é o número de passageiros médio por rota (origem e destino)?


## 19. Qual é o número de passageiros acumulado por ano?


## 20. Qual é o destino mais popular?


## 21. Qual é o destino recebe mais passageiros?


## 22. Qual é o destino mais distante dePDX?


## 23. Qual é o destino tem mais voos por mes?


## 24. Qual é o modelo de avião mais popular?


## 25. Qual é o modelo de avião mais popular por destino?


## 26. Qual é o número médio de motores para cada categoria dehaul\_duration?


## 27. Qual é a estação mais popular para voos?


## 28. Qual é a estação mais popular para voos em cada um dos destinos?


## 29. Qual é a categoria de atrasos mais comum?

In [0]:

flights_qa.show()


## 30. Qual é a categoria de atrasos mais comum por rota?